## 掛載雲端硬碟

In [14]:
# from google.colab import drive
# drive.mount('/content/drive')

## 更改檔案所在路徑


In [15]:
# # Change to your own folder !!!
# %cd /content/drive/MyDrive/your own folder/

## 載入函式庫


In [16]:
import os

import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import datasets, transforms
import numpy as np

from models.resnet import ResNet50

## 超參數設定

In [17]:
DATASET = 'cifar10'
TEST_BATCH_SIZE = 1000
CUDA = True
PRUNE_PERCENT = 0.9 # Change your prune ratio!
WEIGHT_PATH = 'bestmodel/model_best1.pth' # Change to your own folder !!!
PRUNE_PATH = 'bestmodel/model_prune09.pth' # Change to your own folder !!!

## 載入模型

In [18]:
CUDA = CUDA and torch.cuda.is_available()

model = ResNet50(num_classes=10)
if CUDA:
    model.cuda()

if WEIGHT_PATH:
    if os.path.isfile(WEIGHT_PATH):
        checkpoint = torch.load(WEIGHT_PATH)
        best_prec1 = checkpoint['best_prec1']
        model.load_state_dict(checkpoint['state_dict'])
        print('LOADING CHECKPOINT {} @EPOCH={}, BEST_PREC1={}'.format(WEIGHT_PATH,checkpoint['epoch'],best_prec1))

    else:
        print("NO CHECKPOINT FOUND")

print(model)

LOADING CHECKPOINT bestmodel/model_best1.pth @EPOCH=29, BEST_PREC1=0.90420001745224
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (d

## 進行剪枝
#### 計算所有Batch Normalizaiton中的scale factor絕對值大小並排序
#### 利用設定好的PRUNE_PERCENT來取得閥值

In [19]:
total = 0
for m in model.modules():
    if isinstance(m, nn.BatchNorm2d):
        total += m.weight.data.shape[0]

bn = torch.zeros(total)
index = 0
for m in model.modules():
    if isinstance(m, nn.BatchNorm2d):
        size = m.weight.data.shape[0]
        bn[index:(index+size)] = m.weight.data.abs().clone()
        index += size

y, i = torch.sort(bn)

threshold_index = int(total * PRUNE_PERCENT)
threshold = y[threshold_index]


## 根據Batch Normalization Layer資訊建立CONFIG
#### 1. 複製Batch Normalization Layer的weight(也就是scale factor γ)
#### 2. 建立mask，大於threshold的index的值會設成1,小於threshold的值會設成0
#### 3. mask的值加總後，會是剪枝後Layer對應的輸出channel數
#### 4. 最後得到要建立剪枝模型的CONFIG

In [20]:
pruned = 0
cfg = []  #用來建立剪枝網路的CONFIG
cfg_mask = [] #用來幫助剪枝的遮罩

In [21]:
bn_layer_counter = 0
for k, m in enumerate((model.modules())):
    if isinstance(m, nn.BatchNorm2d):
        bn_layer_counter += 1
        weight_copy = m.weight.data.clone()
        mask = weight_copy.abs().gt(threshold).float().cuda() # 大於 threshold 的設為 True (1.0)，其餘為 False(0.0)

        # 注意: 需自行設計處理剩下channel數為0的情況 (e.g. 至少保留3個channel)
        ################################################
        #          請填空          #
        is_bn3_layer = False
        # 第 1 個 BN 層是模型最前面的 bn1 (bn_layer_counter=1)，跳過
        # 之後，BN 層會以 [bn1, bn2, bn3] 重複 (bn_layer_counter=2, 3, 4, ...)
        # 所以計數器在 4, 7, 10, 13... 時 (即 (x-1)%3 == 0)，代表 bn3
        if bn_layer_counter > 1 and (bn_layer_counter - 1) % 3 == 0: # <-- 3. 這是識別 bn3 的新邏輯
            is_bn3_layer = True

        if is_bn3_layer:
            # 這是 bn3，覆蓋 mask，強制設為全 1 (不剪枝)
            mask = torch.ones(m.weight.data.shape).cuda()
            print(f'layer index: {k:d} \t total channel: {mask.shape[0]:d} \t (SKIPPING BN3) \t remaining channel: {int(torch.sum(mask)):d}')
        else:
            # 這是 bn1 或 bn2 (或最前面的 bn1)，使用上面算出的 mask
            
            # 注意: 需自行設計處理剩下channel數為0的情況 (e.g. 至少保留3個channel)
            if int(torch.sum(mask)) < 3:
                _, sorted_idx = weight_copy.abs().sort(descending=True)
                mask.zero_()
                mask[sorted_idx[:3]] = 1.0
        ################################################



        # 處理剪枝後的權重
        m.weight.data.mul_(mask)
        m.bias.data.mul_(mask)
        pruned = pruned + mask.shape[0] - torch.sum(mask)
        cfg.append(int(torch.sum(mask)))    # 記錄每一層 BN 剩下幾個通道
        cfg_mask.append(mask.clone())     # 儲存每層對應的 mask
        print('layer index: {:d} \t total channel: {:d} \t remaining channel: {:d}'.
            format(k, mask.shape[0], int(torch.sum(mask))))

pruned_ratio = pruned/total

print(f'PRUNE RATIO={pruned_ratio}')
print('PREPROCESSING SUCCESSFUL!')

print(f'cfg: {cfg}')


layer index: 2 	 total channel: 64 	 remaining channel: 64
layer index: 8 	 total channel: 64 	 remaining channel: 64
layer index: 10 	 total channel: 64 	 remaining channel: 64
layer index: 12 	 total channel: 256 	 (SKIPPING BN3) 	 remaining channel: 256
layer index: 12 	 total channel: 256 	 remaining channel: 256
layer index: 18 	 total channel: 64 	 remaining channel: 64
layer index: 20 	 total channel: 64 	 remaining channel: 64
layer index: 22 	 total channel: 256 	 (SKIPPING BN3) 	 remaining channel: 256
layer index: 22 	 total channel: 256 	 remaining channel: 256
layer index: 26 	 total channel: 64 	 remaining channel: 64
layer index: 28 	 total channel: 64 	 remaining channel: 64
layer index: 30 	 total channel: 256 	 (SKIPPING BN3) 	 remaining channel: 256
layer index: 30 	 total channel: 256 	 remaining channel: 256
layer index: 35 	 total channel: 128 	 remaining channel: 119
layer index: 37 	 total channel: 128 	 remaining channel: 124
layer index: 39 	 total channel: 51

## 建立剪枝模型

In [22]:
newmodel = ResNet50(num_classes=10, cfg=cfg)
newmodel.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

### 將原本的模型權重複製到剪枝的模型
#### 根據不同層決定要複製什麼權重
###### Batch Normalization Layer
1.   scale factor
2.   bias
3.   running mean
4.   running variance

###### Convolutional Layer
1.   weight

###### Linear Layer
1.   weight
2.   bias




In [23]:
old_modules = list(model.modules())
new_modules = list(newmodel.modules())

layer_id_in_cfg = 0
start_mask = torch.ones(3) #3為input channel(R,G,B)
end_mask = cfg_mask[layer_id_in_cfg]
bn_count = 0

for layer_id in range(len(old_modules)):

    m0 = old_modules[layer_id]
    m1 = new_modules[layer_id]

    if isinstance(m0, nn.BatchNorm2d):
        bn_count += 1

        #### 找出遮罩中非零元素的index ####
        ################################################
        idx1 = np.squeeze(np.argwhere(np.asarray(end_mask.cpu().numpy())))
        if idx1.ndim == 0:
            idx1 = np.expand_dims(idx1, 0)
        ################################################


        #### 複製weight, bias, running mean,and running variance ####
        ################################################
        m1.weight.data = m0.weight.data[idx1].clone()
        m1.bias.data = m0.bias.data[idx1].clone()
        m1.running_mean = m0.running_mean[idx1].clone()
        m1.running_var = m0.running_var[idx1].clone()
        ################################################


    

        layer_id_in_cfg += 1
        start_mask = end_mask.clone()

        #最後一層連接層不做修改
        if layer_id_in_cfg < len(cfg_mask):
            end_mask = cfg_mask[layer_id_in_cfg]


    elif isinstance(m0, nn.Conv2d):
        if isinstance(old_modules[layer_id + 1], nn.BatchNorm2d):
            idx0 = np.squeeze(np.argwhere(np.asarray(start_mask.cpu().numpy())))
            idx1 = np.squeeze(np.argwhere(np.asarray(end_mask.cpu().numpy())))

            #### 複製weight ####
            ################################################
            w = m0.weight.data[:, idx0.tolist(), :, :].clone()
            w = w[idx1.tolist(), :, :, :].clone()
            m1.weight.data = w.clone()
            ################################################




        # downsample 層不用prune
        else:
            m1.weight.data = m0.weight.data.clone()


    elif isinstance(m0, nn.Linear):

        idx0 = np.squeeze(np.argwhere(np.asarray(start_mask.cpu().numpy())))
        if idx0.ndim == 0:
            idx0 = np.expand_dims(idx0, 0)

        #### 複製weight ####
        ################################################
        m1.weight.data = m0.weight.data.clone()
        ################################################
        


        #### 複製bias ####
        m1.bias.data = m0.bias.data.clone()


## 測試函數




In [24]:
def test(model):
    kwargs = {'num_workers': 1, 'pin_memory': True} if CUDA else {}
    test_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10('./data', train=False, download=True, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])),
        batch_size=TEST_BATCH_SIZE, shuffle=True, **kwargs)

    model.eval()
    correct = 0
    with torch.no_grad():
      for data, target in test_loader:
          if CUDA:
              data, target = data.cuda(), target.cuda()
          data, target = Variable(data), Variable(target)
          output = model(data)
          pred = output.data.max(1, keepdim=True)[1]
          correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    print('\nTest set: Accuracy: {}/{} ({:.1f}%)\n'.format(
        correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))
    return correct / float(len(test_loader.dataset))

## 儲存模型並印出結果，以及剪枝後的test acc


In [26]:
torch.save({'cfg': cfg, 'state_dict': newmodel.state_dict()}, PRUNE_PATH)

print(newmodel)
model = newmodel.cuda()
test(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

tensor(0.4725)